# 1. Análises Comparativas

## 1.1. Análises IntraClusters

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import date

class ClusterMetrics:
    """
    Classe para análise estatística de clusters de fundos de investimento.
    Realiza carregamento de dados, preparação de colunas auxiliares e cálculo de métricas agregadas,
    incluindo médias, desvios padrão, proporções e índice de concentração HHI.
    """

    def __init__(self, spark, tipo_fundo: str):
        """
        Inicializa a classe com o objeto Spark e o tipo de fundo.
        :param spark: sessão Spark ativa
        :param tipo_fundo: tipo de fundo a ser analisado (ex: 'fidc', 'fii', 'fip')
        """
        self.spark = spark
        self.tipo_fundo = tipo_fundo.lower()
        self.df = None
        self.final_df = None

    def load_data(self):
        """
        Carrega os dados do cluster para o tipo de fundo especificado.
        Espera-se que a tabela contenha as colunas necessárias para as análises seguintes.
        """
        path = f"desafio_kinea.prospecto_fundos.resultados_cluster_{self.tipo_fundo}"
        self.df = self.spark.table(path)

    def prepare_columns(self):
        """
        Prepara colunas auxiliares para análise, como idade do fundo e flag de chamada de capital IPCA.
        'idade_anos' é calculada como a diferença entre a data atual e a menor data de emissão por CNPJ.
        Cria flag binária para identificar fundos com chamada de capital IPCA.
        """
        w_cnpj = Window.partitionBy("cnpj")
        self.df = self.df.withColumn(
            "idade_anos",
            F.datediff(F.current_date(), F.min("data_emissao").over(w_cnpj)) / 365.25
        )
        self.df = self.df.withColumn(
            "chamada_capital_ipca_flag",
            F.when(
                F.upper(F.coalesce(F.col("chamada_capital_ipca"), F.lit("NÃO"))).like("%SIM%"),
                1
            ).otherwise(0)
        )

    def calculate_aggregates(self):
        """
        Calcula métricas agregadas por cluster, incluindo médias, desvios padrão e índice HHI.
        volume_total_global é o somatório do volume_base_emissao de todos os fundos do tipo analisado.
        Agrega métricas estatísticas por cluster_id_full e calcula o índice de concentração HHI
        para o volume_base_emissao dentro de cada cluster_id_full.
        Junta as métricas agregadas com o HHI e calcula a participação de mercado do cluster.
        """
        volume_total_global = self.df.agg(F.sum("volume_base_emissao").alias("total")).collect()[0]["total"]

        agg_df = self.df.groupBy("cluster_id_full").agg(
            F.countDistinct("cnpj").alias("qtd_fundos"),
            F.round(F.mean("volume_base_emissao"), 2).alias("media_volume_base_emissao"),
            F.round(F.mean("qt_emissoes"), 2).alias("media_qt_emissoes"),
            F.mean("valor_cota_emissao").alias("media_valor_cota_emissao"),
            F.mean("taxa_distribuicao_emissao").alias("media_taxa_distribuicao_emissao"),
            F.mean("quantidade_cotas_totais").alias("media_quantidade_cotas_totais"),
            F.mean("percentual_oferta_institucional").alias("media_percentual_oferta_institucional"),
            F.mean("montante_minimo_emissao").alias("media_montante_minimo_emissao"),
            F.stddev("volume_base_emissao").alias("std_volume_base_emissao"),
            F.stddev("qt_emissoes").alias("std_qt_emissoes"),
            F.stddev("valor_cota_emissao").alias("std_valor_cota_emissao"),
            F.stddev("taxa_distribuicao_emissao").alias("std_taxa_distribuicao_emissao"),
            F.stddev("quantidade_cotas_totais").alias("std_quantidade_cotas_totais"),
            F.stddev("percentual_oferta_institucional").alias("std_percentual_oferta_institucional"),
            F.stddev("montante_minimo_emissao").alias("std_montante_minimo_emissao"),
            F.mean("chamada_capital_ipca_flag").alias("proporcao_chamada_capital_ipca"),
            F.mean("sharpe_ratio").alias("media_sharpe_ratio"),
            F.sum("volume_base_emissao").alias("volume_total_cluster"),
            F.mean("idade_anos").alias("idade_media_anos")
        )

        w_cluster = Window.partitionBy("cluster_id_full")
        df_hhi = self.df.withColumn(
            "prop_volume_cluster",
            F.col("volume_base_emissao") / F.sum("volume_base_emissao").over(w_cluster)
        ).withColumn(
            "hhi_component",
            F.pow(F.col("prop_volume_cluster"), 2)
        )

        hhi_df = df_hhi.groupBy("cluster_id_full").agg(
            F.sum("hhi_component").alias("hhi_volume_base_emissao")
        )

        self.final_df = agg_df.join(hhi_df, on="cluster_id_full", how="left").withColumn(
            "proporcao_participacao_mercado",
            F.col("volume_total_cluster") / F.lit(volume_total_global)
        )

    def run(self):
        """
        Executa o pipeline de análise: carrega dados, prepara colunas e calcula agregados.
        :return: DataFrame final com métricas agregadas por cluster
        """
        self.load_data()
        self.prepare_columns()
        self.calculate_aggregates()
        return self.final_df

In [0]:
tipos_fundo = ["fidc", "fii", "fip"]

# Itera sobre a lista e aplica a classe
resultados = {}
for tipo in tipos_fundo:
    cm = ClusterMetrics(spark, tipo)
    resultados[tipo] = cm.run()

# Visualizando
for tipo in tipos_fundo:
    display(resultados[tipo])

In [0]:
for tipo in tipos_fundo:
    # Recupera o DataFrame de clusters para o tipo de fundo
    df_analise = resultados[tipo]

    # Salva o DataFrame resultante no schema especificado, sobrescrevendo se já existir
    tabela = f"desafio_kinea.prospecto_fundos.analise_cluster_{tipo.lower()}"
    df_analise.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela)

## 1.2. Análise Kinea versus Cluster

Avaliando performance **conjunta** dos fundos KINEA em comparação aos fundos concorrentes:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

class KineaClusterComparisonFull:
    def __init__(self, df):
        """
        Inicializa a classe com o DataFrame base
        :param df: DataFrame contendo todos os fundos e clusters
        """
        self.df = df
        self.final_comp_df = None

    def filter_kinea_clusters(self):
        """Mantém apenas clusters onde a Kinea está presente"""
        clusters_kinea_ids = (
            self.df.filter(F.upper(F.col("tipo_gestor")) == "KINEA")
                   .select("cluster_id")
                   .distinct()
        )
        self.df = self.df.join(clusters_kinea_ids, on="cluster_id", how="inner")

    def prepare_columns(self):
        """Prepara colunas auxiliares"""
        # Idade do fundo em anos (considera primeira emissão por CNPJ)
        w_cnpj = Window.partitionBy("cnpj")
        self.df = self.df.withColumn(
            "idade_anos",
            F.datediff(F.current_date(), F.min("data_emissao").over(w_cnpj)) / 365.25
        )

        # Flag binária chamada de capital IPCA
        self.df = self.df.withColumn(
            "chamada_capital_ipca_flag",
            F.when(
                F.upper(F.coalesce(F.col("chamada_capital_ipca"), F.lit("NÃO"))).like("%SIM%"),
                1
            ).otherwise(0)
        )

        # Sharpe do fundo
        self.df = self.df.withColumn(
            "sharpe_ratio_calc",
            F.when(F.col("volatilidade_historica") != 0, F.col("retorno_acumulado") / F.col("volatilidade_historica")).otherwise(0)
        )

        # Volume total do cluster
        w_cluster = Window.partitionBy("cluster_id")
        self.df = self.df.withColumn(
            "volume_total_cluster",
            F.sum("volume_base_emissao").over(w_cluster)
        )

        # Market share do fundo
        self.df = self.df.withColumn(
            "market_share_fundo",
            (F.col("volume_base_emissao") / F.col("volume_total_cluster")) * 100
        )

    def aggregate_metrics(self):
        """Agrega métricas por cluster e tipo_gestor"""
        metrics = [
            "retorno_acumulado",
            "volatilidade_historica",
            "sharpe_ratio_calc",
            "volume_base_emissao",
            "qt_emissoes",
            "valor_cota_emissao",
            "taxa_distribuicao_emissao",
            "quantidade_cotas_totais",
            "percentual_oferta_institucional",
            "montante_minimo_emissao",
            "chamada_capital_ipca_flag",
            "idade_anos"
        ]

        agg_exprs = [F.mean(m).alias(m) for m in metrics]
        agg_exprs += [
            F.sum("volume_base_emissao").alias("volume_total_gestor"),
            F.sum("market_share_fundo").alias("market_share_gestor"),
            F.countDistinct("cnpj").alias("qtd_fundos")
        ]

        self.agg_metrics = self.df.groupBy("cluster_id", "tipo_gestor").agg(*agg_exprs)

    def pivot_and_compare(self):
        """Pivot Kinea vs Concorrentes e organiza lado a lado"""
        # Pivot
        pivot_df = self.agg_metrics.groupBy("cluster_id").pivot("tipo_gestor").agg(
            *[F.first(c).alias(c) for c in self.agg_metrics.columns if c not in ["cluster_id", "tipo_gestor"]]
        )

        metrics = [
            "qtd_fundos",
            "retorno_acumulado",
            "volatilidade_historica",
            "sharpe_ratio_calc",
            "volume_base_emissao",
            "qt_emissoes",
            "valor_cota_emissao",
            "taxa_distribuicao_emissao",
            "quantidade_cotas_totais",
            "percentual_oferta_institucional",
            "montante_minimo_emissao",
            "chamada_capital_ipca_flag",
            "idade_anos",
            "volume_total_gestor",
            "market_share_gestor"
        ]

        # Calcula diferenças Kinea vs Concorrente
        for m in metrics:
            pivot_df = pivot_df.withColumn(
                f"dif_{m}",
                F.coalesce(F.col(f"KINEA_{m}"), F.lit(0)) - F.coalesce(F.col(f"CONCORRENTE_{m}"), F.lit(0))
            )

        # Organiza colunas lado a lado
        cols_lado_a_lado = ["cluster_id"]
        for m in metrics:
            cols_lado_a_lado += [f"KINEA_{m}", f"CONCORRENTE_{m}", f"dif_{m}"]

        self.final_comp_df = pivot_df.filter(F.col("KINEA_qtd_fundos").isNotNull()).select(*cols_lado_a_lado)

    def run(self):
        """Executa todo o pipeline"""
        self.filter_kinea_clusters()
        self.prepare_columns()
        self.aggregate_metrics()
        self.pivot_and_compare()
        return self.final_comp_df

In [0]:

tipos_fundo = ["fidc", "fii", "fip"]

# Itera sobre a lista e aplica a classe
resultados = {}
for tipo in tipos_fundo:
    # Carrega o DataFrame do tipo de fundo
    df_fundo = spark.table(f"desafio_kinea.prospecto_fundos.resultados_cluster_{tipo}")
    
    # Aplica a classe
    kcc = KineaClusterComparisonFull(df_fundo)
    resultados[tipo] = kcc.run()

# Visualizando
for tipo in tipos_fundo:
    display(resultados[tipo])

In [0]:
for tipo in tipos_fundo:
    # Recupera o DataFrame de clusters para o tipo de fundo
    df_analise = resultados[tipo]

    # Salva o DataFrame resultante no schema especificado, substituindo se já existir
    tabela = f"desafio_kinea.prospecto_fundos.analise_cluster_kinea_conjunto_{tipo.lower()}"
    df_analise.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela)

Avaliando performance **individual** dos fundos KINEA em comparação aos fundos concorrentes:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

class KineaPerformance:
    """
    Classe para análise de performance individual dos fundos KINEA em relação à média dos clusters.
    Realiza carregamento de dados, preparação de colunas auxiliares, cálculo de métricas operacionais e financeiras,
    comparação das métricas dos fundos KINEA com a média do cluster e cálculo do market share.
    """

    def __init__(self, spark, tipo_fundo: str):
        """
        Inicializa a classe com o objeto Spark e o tipo de fundo.
        :param spark: sessão Spark ativa
        :param tipo_fundo: tipo de fundo a ser analisado (ex: 'fidc', 'fii', 'fip')
        """
        self.spark = spark
        self.tipo_fundo = tipo_fundo.lower()
        self.df = None
        self.result_df = None

    def load_data(self):
        """
        Carrega os dados do cluster para o tipo de fundo especificado.
        """
        path = f"desafio_kinea.prospecto_fundos.resultados_cluster_{self.tipo_fundo}"
        self.df = self.spark.table(path)

    def prepare_columns(self):
        """
        Prepara colunas auxiliares para análise:
        - Idade do fundo em anos (diferença entre hoje e primeira emissão por CNPJ)
        - Flag binária para chamada de capital IPCA
        - Sharpe do fundo (retorno/volatilidade), se as colunas existirem
        """
        w_cnpj = Window.partitionBy("cnpj")
        self.df = self.df.withColumn(
            "idade_anos",
            F.datediff(F.current_date(), F.min("data_emissao").over(w_cnpj)) / 365.25
        )
        self.df = self.df.withColumn(
            "chamada_capital_ipca_flag",
            F.when(
                F.upper(F.coalesce(F.col("chamada_capital_ipca"), F.lit("NÃO"))).like("%SIM%"),
                1
            ).otherwise(0)
        )
        if "retorno_acumulado" in self.df.columns and "volatilidade_historica" in self.df.columns:
            self.df = self.df.withColumn(
                "sharpe_ratio_calc",
                F.when(F.col("volatilidade_historica") != 0, F.col("retorno_acumulado") / F.col("volatilidade_historica")).otherwise(0)
            )
        else:
            self.df = self.df.withColumn("sharpe_ratio_calc", F.lit(0))

    def calculate_performance(self):
        """
        Calcula métricas operacionais e financeiras dos fundos KINEA e compara com a média do cluster.
        - Calcula médias por cluster
        - Junta médias do cluster com fundos KINEA
        - Calcula diferenças KINEA vs média do cluster
        - Calcula market share do fundo
        """
        metrics = [
            "volume_base_emissao",
            "qt_emissoes",
            "valor_cota_emissao",
            "taxa_distribuicao_emissao",
            "quantidade_cotas_totais",
            "percentual_oferta_institucional",
            "montante_minimo_emissao",
            "chamada_capital_ipca_flag",
            "sharpe_ratio_calc",
            "idade_anos",
        ]

        financial_metrics = []
        if "retorno_acumulado" in self.df.columns:
            financial_metrics.append("retorno_acumulado")
        if "volatilidade_historica" in self.df.columns:
            financial_metrics.append("volatilidade_historica")

        all_metrics = metrics + financial_metrics

        cluster_metrics = self.df.groupBy("cluster_id_full").agg(
            *[F.mean(m).alias(f"media_cluster_{m}") for m in all_metrics],
            F.sum("volume_base_emissao").alias("volume_total_cluster")
        )

        kinea_fundos = self.df.filter(F.upper(F.col("tipo_gestor")) == "KINEA")

        joined = kinea_fundos.join(cluster_metrics, on="cluster_id_full", how="left")

        for m in all_metrics:
            joined = joined.withColumn(
                f"{m}_diff",
                F.coalesce(F.col(m), F.lit(0)) - F.coalesce(F.col(f"media_cluster_{m}"), F.lit(0))
            )

        joined = joined.withColumn(
            "market_share_fundo",
            (F.col("volume_base_emissao") / F.col("volume_total_cluster") * 100)
        ).withColumn(
            "dif_market_share",
            F.col("market_share_fundo") - (F.col("media_cluster_volume_base_emissao") / F.col("volume_total_cluster") * 100)
        )

        select_cols = ["cnpj", "nome_fundo", "cluster_id_full"]
        for m in all_metrics:
            select_cols += [m, f"media_cluster_{m}", f"{m}_diff"]
        select_cols += ["volume_total_cluster", "market_share_fundo", "dif_market_share"]

        self.result_df = joined.select(*select_cols)

    def run(self):
        """
        Executa o pipeline de análise: carrega dados, prepara colunas e calcula performance.
        :return: DataFrame final com métricas individuais dos fundos KINEA e comparação com o cluster
        """
        self.load_data()
        self.prepare_columns()
        self.calculate_performance()
        return self.result_df

In [0]:
tipos_fundo = ["fidc", "fii", "fip"]
resultados = {}

for tipo in tipos_fundo:
    # Cria instância da classe KineaPerformance para o tipo de fundo
    kperf = KineaPerformance(spark, tipo)
    # Executa o pipeline e salva o DataFrame no dicionário
    resultados[tipo] = kperf.run()

# Visualizar resultados
for tipo in tipos_fundo:
    print(f"=== {tipo.upper()} ===")
    display(resultados[tipo])


In [0]:
for tipo in tipos_fundo:
    # Recupera o DataFrame de clusters para o tipo de fundo
    df_analise = resultados[tipo]

    # Salva o DataFrame resultante no schema especificado, sobrescrevendo se já existir
    tabela = f"desafio_kinea.prospecto_fundos.analise_cluster_kinea_individual_{tipo.lower()}"
    df_analise.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela)

## 1.2.1. Análises Gráficas KINEA

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Carregar os dados
df = spark.table("desafio_kinea.prospecto_fundos.resultados_cluster_fidc").toPandas()

# Filtrar dados da Kinea
kinea = df[df['tipo_gestor'].str.upper() == 'KINEA']

# Variáveis para análise
variaveis = [
    "valor_cota_emissao",
    "quantidade_cotas_totais",
    "montante_minimo_emissao",
    "volume_base_emissao", 
    "taxa_distribuicao_emissao"
]

# Configurações de plot
plt.style.use('seaborn')
sns.set_palette("pastel")

for var in variaveis:
    plt.figure(figsize=(15, 6))
    
    # --- BOXPLOT ---
    plt.subplot(1, 2, 1)
    
    # Boxplot com violino
    sns.violinplot(x=var, data=df, color='lightgray', inner=None)
    sns.boxplot(x=var, data=df, width=0.1, color='white', 
                showcaps=True, fliersize=3)
    
    # Destacar Kinea
    for val in kinea[var]:
        plt.axvline(val, color='red', linestyle='-', linewidth=2, alpha=0.7)
    
    # Estatísticas descritivas
    mean_val = df[var].mean()
    median_val = df[var].median()
    plt.axvline(mean_val, color='green', linestyle='--', linewidth=2, label=f'Média: {mean_val:.2f}')
    plt.axvline(median_val, color='blue', linestyle=':', linewidth=2, label=f'Mediana: {median_val:.2f}')
    
    plt.title(f'Distribuição de {var}\nPosição da Kinea', fontsize=12, pad=20)
    plt.xlabel(var)
    plt.legend()
    
    # --- HISTOGRAMA ---
    plt.subplot(1, 2, 2)
    
    # Histograma com KDE
    sns.histplot(df[var], bins=30, kde=True, color='skyblue', alpha=0.7)
    
    # Linhas para Kinea
    for val in kinea[var]:
        plt.axvline(val, color='red', linestyle='-', linewidth=2, alpha=0.7, label='Kinea')
    
    # Linhas para estatísticas
    plt.axvline(mean_val, color='green', linestyle='--', linewidth=2, label='Média')
    plt.axvline(median_val, color='blue', linestyle=':', linewidth=2, label='Mediana')
    
    # Ajustar escala se necessário
    if df[var].skew() > 1:
        plt.xscale('log')
        plt.xlabel(f'{var} (escala log)')
    
    plt.title(f'Histograma de {var}', fontsize=12, pad=20)
    plt.xlabel(var)
    plt.ylabel('Frequência')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

## 1.3. Potenciais Novas Emissões

In [0]:
tipos_fundo = ["fidc", "fii", "fip"]

for tipo in tipos_fundo:
    df = spark.table(f"desafio_kinea.prospecto_fundos.resultados_cluster_{tipo}")
    fundos_potenciais = df.filter(F.col("potencial_nova_emissao") == True)
    display(fundos_potenciais)